In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
from collections import defaultdict

import sqlite3
import sys
import time
import math
import tqdm

from glicko2 import Player

## Loading SQLite Database into Pandas DataFrames

The following code connects to an SQLite database (`melee_player_database.db`) and converts each table within the database into a pandas DataFrame. The DataFrames will be stored in a dictionary, where each key corresponds to the table name with `_df` appended, and the values are the respective DataFrames.

### Steps:

1. **Database Connection**: We use the `sqlite3` library to connect to the SQLite database file.
2. **Retrieve Table Names**: A query retrieves all the table names in the database.
3. **Convert Tables to DataFrames**: For each table:
   - The table is loaded into a pandas DataFrame using `pd.read_sql()`.
   - We check each column to see if any data is JSON-formatted (lists or dictionaries). If so, we convert these columns from strings into their corresponding Python objects using `json.loads()`.
4. **Store DataFrames**: The DataFrames are stored in a dictionary, where the key is the table name with a `_df` suffix, and the value is the DataFrame.
5. **Database Connection Closed**: Once all tables are loaded into DataFrames, the database connection is closed.

### Example:
If the database contains a table named `players`, the corresponding DataFrame will be stored in the dictionary with the key `players_df`, and can be accessed as:

```python
players_df = dfs['players_df']


In [2]:
# Function to get the table names
def get_table_names(conn):
    query = "SELECT name FROM sqlite_master WHERE type='table';"
    return pd.read_sql(query, conn)['name'].tolist()

# Function to load tables into DataFrames
def load_tables_to_dfs(conn):
    table_names = get_table_names(conn)
    dataframes = {}
    
    for table in table_names:
        # Load table into a DataFrame
        df = pd.read_sql(f"SELECT * FROM {table}", conn)
        
        # Detect and convert JSON formatted columns (if any)
        for col in df.columns:
            # Check if any entry in the column is a valid JSON (list or dictionary)
            if df[col].apply(lambda x: isinstance(x, str)).all():
                try:
                    # Try parsing the column as JSON
                    df[col] = df[col].apply(lambda x: json.loads(x) if pd.notnull(x) else x)
                except (json.JSONDecodeError, TypeError):
                    # If it fails, skip the column
                    pass
        
        # Store the DataFrame with table name + '_df'
        dataframes[f"{table}_df"] = df
        
    return dataframes

# Check if the flag variable exists in the global scope so that this code does not run twice
if 'cell_has_run' not in globals():
    path = "../data/melee_player_database.db"
    
    # Connect to the database
    conn = sqlite3.connect(path)

    # Convert each table into a DataFrame
    dfs = load_tables_to_dfs(conn)

    # Close the connection
    conn.close()

    # Now, you have a dictionary 'dfs' where each key is the table name with '_df' suffix and value is the corresponding DataFrame.
    # For example, to access the DataFrame for a table called 'players':
    # players_df = dfs['players_df']

    dfs['tournament_info_df']['start'] = pd.to_datetime(dfs['tournament_info_df']['start'], unit='s')
    dfs['tournament_info_df']['end'] = pd.to_datetime(dfs['tournament_info_df']['end'], unit='s')

    
    # Set the flag to indicate that the cell has been run
    cell_has_run = True

### Here we adjust the data types of the dataframes so that they are the correct type. (This will be updated as needed.)

In [3]:
dfs['sets_df']['best_of'] = dfs['sets_df']['best_of'].fillna(0).astype(int) 

### Here we make dataframes that we will use and print the head.

The integers in 'characters' count the number of games the player has played that character. (We verify this for Zain below.)

In [4]:
players_df = dfs['players_df']
players_df.head()

,game,player_id,tag,all_tags,prefixes,social,country,state,region,c_country,c_state,c_region,placings,characters,alias
0,melee,Rishi,Rishi,[Rishi],[],{'twitter': []},,,,None,None,None,[{'key': 'mdva-invitational-2017-(challonge-mi...,,None
1,melee,15634,lloD,"[lloD, VGz | lloD, Llod]",[],{'twitter': ['lloD74']},United States,VA,,US,CA,Laurel,[{'key': 'mdva-invitational-2017-(challonge-mi...,"{'melee/peach': 1089, 'melee/falco': 1, 'melee...",None
2,melee,6126,Zain,"[Zain, DontTestMe]",[PG],{'twitter': ['PG_Zain']},United States,VA,,US,CA,Los Angeles,[{'key': 'mdva-invitational-2017-(challonge-mi...,"{'melee/marth': 1065, 'melee/pichu': 1, 'melee...",DontTestMe
3,melee,Chu,Chu,[Chu],[],{'twitter': []},,,,None,None,None,[{'key': 'mdva-invitational-2017-(challonge-mi...,,None
4,melee,5620,Junebug,"[Junebug, LS | VGz Junebug]",[],{'twitter': ['arJunebug']},United States,VA,,US,VA,Richmond,[{'key': 'mdva-invitational-2017-(challonge-mi...,"{'melee/sheik': 46, 'melee/falco': 4, 'melee/g...",None


In [5]:
ranking_df = dfs['ranking_df']
ranking_df.head()

,game,ranking_name,priority,region,seasons,tournaments,icon
0,melee,SSBMRank,0,world,"[2015, 2016, 2017, 2018, 2019]",[],miom


In [6]:
ranking_seasons_df = dfs['ranking_seasons_df']
ranking_seasons_df.head()

,game,ranking_name,season,start,end,total,by_id,by_placing,final,name
0,melee,SSBMRank,2015,1420070400,1451606399,100,"{'6189': 1, '1004': 2, '4465': 3, '1000': 4, '...","{'1': '6189', '2': '1004', '3': '4465', '4': '...",0,None
1,melee,SSBMRank,2016,1451606400,1483228799,100,"{'6189': 1, '1004': 2, '1000': 3, '1003': 4, '...","{'1': '6189', '2': '1004', '3': '1000', '4': '...",0,None
2,melee,SSBMRank,2017,1483228800,1514764799,100,"{'1004': 1, '6189': 2, '1000': 3, '1003': 4, '...","{'1': '1004', '2': '6189', '3': '1000', '4': '...",0,None
3,melee,SSBMRank,2018,1514793600,1546329600,100,"{'1004': 1, '6189': 2, '4465': 3, '15990': 4, ...","{'1': '1004', '2': '6189', '3': '4465', '4': '...",0,None
4,melee,SSBMRank,2019,1546329600,1577836800,100,"{'1004': 1, '4465': 2, '1000': 3, '16342': 4, ...","{'1': '1004', '2': '4465', '3': '1000', '4': '...",0,None


In [7]:
sets_df = dfs['sets_df']
print(f'{sets_df[sets_df['game_data'].apply(lambda x: len(x) > 0)].shape[0] / sets_df.shape[0]:0.01%} percent of sets have some game data')
sets_df.head()

32.9% percent of sets have some game data


,key,game,tournament_key,winner_id,p1_id,p2_id,p1_score,p2_score,location_names,bracket_name,bracket_order,set_order,best_of,game_data
0,104675843,melee,mdva-invitational-2017-(challonge-mirror),5620,5620,Chillin,3,1,"[R1, Round 1, Round 1]",,1,A,5,[]
1,104675844,melee,mdva-invitational-2017-(challonge-mirror),Aglet,15634,Aglet,2,3,"[R1, Round 1, Round 1]",,1,B,5,[]
2,104675845,melee,mdva-invitational-2017-(challonge-mirror),6126,6126,1097,3,0,"[R1, Round 1, Round 1]",,1,C,5,[]
3,104675846,melee,mdva-invitational-2017-(challonge-mirror),1069,Chu,1069,0,3,"[R1, Round 1, Round 1]",,1,D,5,[]
4,104675847,melee,mdva-invitational-2017-(challonge-mirror),Rishi,Jerry,Rishi,1,3,"[R1, Round 1, Round 1]",,1,E,5,[]


In [8]:
tournament_info_df = dfs['tournament_info_df']
tournament_info_df.head()

,game,key,cleaned_name,source,tournament_name,tournament_event,season,rank,start,end,country,state,city,entrants,placings,losses,bracket_types,online,lat,lng
0,melee,mdva-invitational-2017-(challonge-mirror),MDVA Invitational 2017 (Challonge Mirror),challonge,https://challonge.com/mdva_invitational_2017,,17,,2017-11-26 08:05:11,2017-11-26 08:48:09,US,VA,Fall's Church,10,"[[Rishi, 1], [15634, 3], [6126, 4], [Chu, 8], ...",{},b'{}',0,NaN,NaN
1,melee,s@sh7,S@SH7,challonge,https://challonge.com/sash7,,17,,2017-06-13 10:27:01,2017-06-13 10:27:01,US,MI,Ann Arbor,92,[],{},b'{}',0,NaN,NaN
2,melee,slippi-champions-league-week-1__melee-singles,Slippi Champions League Week 1,pgstats,slippi-champions-league-week-1,melee-singles,20,,2020-10-11 14:00:00,2020-10-11 14:00:00,,,,20,"[[1000, 1], [6126, 2], [4107, 3], [19554, 3], ...",{},b'{}',1,0.0,0.0
3,melee,slippi-champions-league-week-2__melee-singles,Slippi Champions League Week 2,pgstats,slippi-champions-league-week-2,melee-singles,20,,2020-10-18 14:00:00,2020-10-18 14:00:00,,,,20,"[[6126, 1], [4107, 2], [1000, 3], [19554, 3], ...",{},b'{}',1,0.0,0.0
4,melee,slippi-champions-league-week-3__melee-singles,Slippi Champions League Week 3,pgstats,slippi-champions-league-week-3,melee-singles,20,,2020-10-25 14:00:00,2020-10-25 14:00:00,,,,20,"[[6126, 1], [3359, 2], [19554, 3], [4107, 3], ...",{},b'{}',1,0.0,0.0


In [9]:
total_character_counts = {}

# Iterate over the 'characters' column
for idx, row in players_df.iterrows():
    char_dict = row['characters']
    if isinstance(char_dict, dict):
        for character, count in char_dict.items():
            if character in total_character_counts:
                total_character_counts[character] += count
            else:
                total_character_counts[character] = count
    else:
        # Handle cases where 'characters' is empty or NaN
        continue

# Now total_character_counts contains the total counts per character

# Sort the characters by total counts in decreasing order
sorted_characters = sorted(total_character_counts.items(), key=lambda x: x[1], reverse=True)
sorted_character_keys = [character[0] for character in sorted_characters]

In [10]:
sorted_tournament_info_df = tournament_info_df.sort_values('end').reset_index(drop=True)
first_date = sorted_tournament_info_df.iloc[0]['start']

game_data_sets_df= sets_df[sets_df['game_data'].apply(lambda x: len(x) > 0)]
seen = set()
tournament_keys = [key for key in game_data_sets_df['tournament_key'] if not (key in seen or seen.add(key))]
# Filter tournament_info_df to include only matching tournament keys
game_data_tournaments_df = tournament_info_df[
    tournament_info_df['key'].isin(tournament_keys)
].sort_values('end').reset_index(drop=True)


print(game_data_sets_df.head(1))
print(f"{game_data_sets_df.iloc[0:1]['game_data'].values[0]}")
print(first_date)

      key   game             tournament_key winner_id p1_id p2_id  p1_score  \
11398      melee  shine-2018__melee-singles      1004  1004  1032         3   

       p2_score                    location_names bracket_name bracket_order  \
11398         0  [W1, Winners 1, Winners Round 1]       Top 64             3   

      set_order  best_of                                          game_data  
11398         A        3  [{'loser_char': 'melee/peach', 'winner_score':...  
[{'loser_char': 'melee/peach', 'winner_score': 1, 'winner_id': 1004, 'loser_id': 1032, 'winner_char': 'melee/jigglypuff', 'loser_score': 0, 'stage': 'Battlefield'}, {'loser_char': 'melee/peach', 'winner_score': 2, 'winner_id': 1004, 'loser_id': 1032, 'winner_char': 'melee/jigglypuff', 'loser_score': 0, 'stage': "Yoshi's Story"}, {'loser_char': 'melee/peach', 'winner_score': 2, 'winner_id': 1004, 'loser_id': 1032, 'winner_char': 'melee/jigglypuff', 'loser_score': 0, 'stage': "Yoshi's Story"}]
2015-01-11 14:16:13


In [11]:
tau = .7

def process_tournament(tournament_key, player_ratings_matchup_df, tournament_info_df, sets_df):
    """
    Optimized function to process a tournament and update character-specific player ratings.
    """
    # Get the sets for this tournament
    tournament_sets_df = sets_df[sets_df['tournament_key'] == tournament_key]

    # Filter out matches without game data
    tournament_sets_df = tournament_sets_df[
        tournament_sets_df['game_data'].apply(lambda x: isinstance(x, list) and len(x) > 0)
    ]

    if tournament_sets_df.empty:
        print(f"No game data for tournament {tournament_key}.")
        return player_ratings_matchup_df

    # Explode 'game_data' to get one row per game
    tournament_sets_df = tournament_sets_df.explode('game_data').reset_index(drop=True)

    # Normalize 'game_data' dictionaries into separate columns
    game_data_df = pd.json_normalize(tournament_sets_df['game_data'])

    # Remove overlapping columns before concatenation
    overlapping_columns = tournament_sets_df.columns.intersection(game_data_df.columns)
    game_data_df = game_data_df.drop(columns=overlapping_columns)

    # Concatenate the game_data_df with tournament_sets_df
    tournament_games_df = pd.concat(
        [tournament_sets_df.reset_index(drop=True), game_data_df.reset_index(drop=True)], axis=1
    )

    # Remove duplicate columns if any
    tournament_games_df = tournament_games_df.loc[:, ~tournament_games_df.columns.duplicated()]

    # Adjust column names based on actual data
    winner_games = tournament_games_df[['winner_id', 'loser_id', 'winner_char', 'loser_char']]
    winner_games = winner_games.rename(columns={
        'winner_id': 'player_id',
        'loser_id': 'opponent_id',
        'winner_char': 'player_char',
        'loser_char': 'opponent_char'
    })
    winner_games['outcome'] = 1  # Winner

    loser_games = tournament_games_df[['loser_id', 'winner_id', 'loser_char', 'winner_char']]
    loser_games = loser_games.rename(columns={
        'loser_id': 'player_id',
        'winner_id': 'opponent_id',
        'loser_char': 'player_char',
        'winner_char': 'opponent_char'
    })
    loser_games['outcome'] = 0  # Loser

    # Combine the data
    player_games_df = pd.concat([winner_games, loser_games], ignore_index=True)

    # Remove duplicate columns if any
    player_games_df = player_games_df.loc[:, ~player_games_df.columns.duplicated()]

    # Filter out games with missing data
    player_games_df.dropna(
        subset=['player_char', 'opponent_char', 'player_id', 'opponent_id'], inplace=True
    )

    # Ensure characters are recognized
    player_games_df = player_games_df[
        player_games_df['player_char'].isin(sorted_character_keys) &
        player_games_df['opponent_char'].isin(sorted_character_keys)
    ]

    # Check if player_games_df is empty
    if player_games_df.empty:
        print(f"No valid player games data for tournament {tournament_key} after filtering.")
        return player_ratings_matchup_df

    # Get the tournament end date
    end_date_series = tournament_info_df.loc[
        tournament_info_df['key'] == tournament_key, 'end'
    ]
    if end_date_series.empty:
        print(f"No end date found for tournament {tournament_key}.")
        return player_ratings_matchup_df
    end_date = pd.to_datetime(end_date_series.values[0])

    # Ensure all players are in player_ratings_matchup_df
    tournament_player_ids = pd.unique(
        player_games_df[['player_id', 'opponent_id']].values.ravel('K')
    )

    existing_player_ids = player_ratings_matchup_df['player_id'].tolist()
    new_players = [pid for pid in tournament_player_ids if pid not in existing_player_ids]

    # Collect new player rows in a list
    new_rows = []

    for player_id in new_players:
        new_row = {
            'player_id': player_id,
            'dates_dict': {},          # Empty dict
            'rating_history_dict': {}, # Empty dict
            'rd_history_dict': {},     # Empty dict
            'game_count_dict': {},     # Empty dict
            'glicko2_dict': {}         # Empty dict
        }
        new_rows.append(new_row)

    # Concatenate once outside the loop if there are new players
    if new_rows:
        new_players_df = pd.DataFrame(new_rows)
        player_ratings_matchup_df = pd.concat([player_ratings_matchup_df, new_players_df], ignore_index=True)

    # Update player_index_map
    player_index_map = {player_id: idx for idx, player_id in enumerate(player_ratings_matchup_df['player_id'])}

    # Prepare data for grouping
    player_games_df['player_index'] = player_games_df['player_id'].map(player_index_map)
    player_games_df['opponent_index'] = player_games_df['opponent_id'].map(player_index_map)

    # Filter out games where players are not in the player_ratings_matchup_df
    player_games_df.dropna(subset=['player_index', 'opponent_index'], inplace=True)
    player_games_df['player_index'] = player_games_df['player_index'].astype(int)
    player_games_df['opponent_index'] = player_games_df['opponent_index'].astype(int)

    # Check if player_games_df is empty
    if player_games_df.empty:
        # print(f"No valid player games data for tournament {tournament_key} after mapping indices.")
        return player_ratings_matchup_df

    # Collect game counts
    game_counts = player_games_df.groupby(
        ['player_index', 'player_char', 'opponent_char']
    ).size().reset_index(name='game_count')

    # Update game counts in the DataFrame
    for row in game_counts.itertuples():
        idx = row.player_index
        p_char = row.player_char
        o_char = row.opponent_char
        count = row.game_count

        # Access or initialize game_count_dict
        game_count_dict = player_ratings_matchup_df.at[idx, 'game_count_dict']
        if p_char not in game_count_dict:
            game_count_dict[p_char] = {}
        if o_char not in game_count_dict[p_char]:
            game_count_dict[p_char][o_char] = 0
        game_count_dict[p_char][o_char] += count

        # Update the DataFrame entry
        player_ratings_matchup_df.at[idx, 'game_count_dict'] = game_count_dict

    # Collect opponent ratings, RDs, and outcomes
    def get_opponent_rating_rd(row):
        try:
            opp_idx = row['opponent_index']
            opp_char = row['opponent_char']
            player_char = row['player_char']

            # Check for missing data
            if pd.isna(opp_idx) or pd.isna(opp_char) or pd.isna(player_char):
                return pd.Series({'opponent_rating': np.nan, 'opponent_rd': np.nan})

            opp_idx = int(opp_idx)

            # Access or initialize opponent's glicko2_dict
            opp_glicko2_dict = player_ratings_matchup_df.at[opp_idx, 'glicko2_dict']
            if opp_char not in opp_glicko2_dict:
                opp_glicko2_dict[opp_char] = {}
            if player_char not in opp_glicko2_dict[opp_char]:
                opp_glicko2_dict[opp_char][player_char] = Player()
                opp_glicko2_dict[opp_char][player_char]._tau = tau
            # print(opp_glicko2_dict[opp_char][player_char]._tau)

            opponent_glicko = opp_glicko2_dict[opp_char][player_char]

            # Update the DataFrame entry
            player_ratings_matchup_df.at[opp_idx, 'glicko2_dict'] = opp_glicko2_dict

            return pd.Series({
                'opponent_rating': opponent_glicko.getRating(),
                'opponent_rd': opponent_glicko.getRd()
            })
        except Exception as e:
            # Log the exception and return NaN values
            print(f"Error in get_opponent_rating_rd for tournament {tournament_key}: {e}")
            return pd.Series({'opponent_rating': np.nan, 'opponent_rd': np.nan})

    # Apply the function and assign the new columns directly
    opponent_ratings_rds = player_games_df.apply(get_opponent_rating_rd, axis=1)

    # Check if opponent_ratings_rds contains the required columns
    if opponent_ratings_rds.empty or not set(['opponent_rating', 'opponent_rd']).issubset(opponent_ratings_rds.columns):
        print(f"Warning: opponent_ratings_rds is empty or missing required columns for tournament {tournament_key}")
        return player_ratings_matchup_df

    player_games_df[['opponent_rating', 'opponent_rd']] = opponent_ratings_rds[['opponent_rating', 'opponent_rd']]

    # Drop rows with NaN opponent ratings
    player_games_df.dropna(subset=['opponent_rating', 'opponent_rd'], inplace=True)

    # Check if player_games_df is empty after dropping NaNs
    if player_games_df.empty:
        print(f"No valid opponent ratings data for tournament {tournament_key} after dropping NaNs.")
        return player_ratings_matchup_df

    # Group data for update_player
    grouped = player_games_df.groupby(['player_index', 'player_char', 'opponent_char'])

    # Update ratings
    for (player_index, player_char, opponent_char), group in grouped:
        p_row = player_ratings_matchup_df.loc[player_index]

        # Access or initialize glicko2_dict
        glicko2_dict = p_row['glicko2_dict']
        if player_char not in glicko2_dict:
            glicko2_dict[player_char] = {}
        if opponent_char not in glicko2_dict[player_char]:
            glicko2_dict[player_char][opponent_char] = Player()
            glicko2_dict[player_char][opponent_char]._tau = tau
        p_vs_char_glicko = glicko2_dict[player_char][opponent_char]

        opponent_ratings = group['opponent_rating'].tolist()
        opponent_rds = group['opponent_rd'].tolist()
        outcomes = group['outcome'].tolist()

        # Update Glicko-2 rating
        p_vs_char_glicko.update_player(opponent_ratings, opponent_rds, outcomes)

        # Access or initialize history dicts
        dates_dict = p_row['dates_dict']
        rating_history_dict = p_row['rating_history_dict']
        rd_history_dict = p_row['rd_history_dict']

        if player_char not in dates_dict:
            dates_dict[player_char] = {}
            rating_history_dict[player_char] = {}
            rd_history_dict[player_char] = {}
        if opponent_char not in dates_dict[player_char]:
            dates_dict[player_char][opponent_char] = []
            rating_history_dict[player_char][opponent_char] = []
            rd_history_dict[player_char][opponent_char] = []

        dates_dict[player_char][opponent_char].append(end_date)
        rating_history_dict[player_char][opponent_char].append(p_vs_char_glicko.getRating())
        rd_history_dict[player_char][opponent_char].append(p_vs_char_glicko.getRd())

        # Update the DataFrame entries
        player_ratings_matchup_df.at[player_index, 'glicko2_dict'] = glicko2_dict
        player_ratings_matchup_df.at[player_index, 'dates_dict'] = dates_dict
        player_ratings_matchup_df.at[player_index, 'rating_history_dict'] = rating_history_dict
        player_ratings_matchup_df.at[player_index, 'rd_history_dict'] = rd_history_dict

    # Return the updated DataFrame
    return player_ratings_matchup_df


In [12]:
# process_tournament(tournament_key, player_ratings_matchup_df, tournament_info_df, sets_df)
# Initialize player ratings DataFrame
player_ratings_matchup_df = pd.DataFrame(columns=[
    'player_id',
    'dates_dict',          # {player_char: {opponent_char: [dates]}}
    'rating_history_dict', # {player_char: {opponent_char: [ratings]}}
    'rd_history_dict',     # {player_char: {opponent_char: [RDs]}}
    'game_count_dict',     # {player_char: {opponent_char: game_count}}
    'glicko2_dict'         # {player_char: {opponent_char: Player()}}
])

# Loop over tournaments
for idx, tournament in tqdm.tqdm(game_data_tournaments_df.iterrows(), total=game_data_tournaments_df.shape[0]):
    player_ratings_matchup_df = process_tournament(tournament['key'], player_ratings_matchup_df, tournament_info_df, sets_df)

  0%|          | 32/16536 [00:04<23:58, 11.47it/s] 

No valid player games data for tournament lanets-2017-feat-rb-tsm-leffen__melee-singles after filtering.


  0%|          | 48/16536 [00:05<23:38, 11.63it/s]

No valid player games data for tournament sweet-28-a-super-smash-bros-melee-event__melee-singles after filtering.


  0%|          | 60/16536 [00:06<25:20, 10.84it/s]

...

100%|█████████▉| 16510/16536 [40:41<00:03,  6.98it/s]

No valid player games data for tournament smash-ntnu-gl-shaugen-191__melee-singles after filtering.


100%|█████████▉| 16527/16536 [40:44<00:01,  7.76it/s]

No valid player games data for tournament liga-melee-br__esq-4 after filtering.


100%|██████████| 16536/16536 [40:46<00:00,  6.76it/s]


In [13]:
player_ratings_matchup_df.head()

,player_id,dates_dict,rating_history_dict,rd_history_dict,game_count_dict,glicko2_dict
0,4643,{'melee/jigglypuff': {'melee/falco': [2015-04-...,{'melee/jigglypuff': {'melee/falco': [1799.625...,{'melee/jigglypuff': {'melee/falco': [227.7354...,"{'melee/jigglypuff': {'melee/falco': 3, 'melee...",{'melee/jigglypuff': {'melee/zelda': <glicko2....
1,4005,{'melee/mario': {'melee/falco': [2015-04-13 05...,{'melee/mario': {'melee/falco': [1799.62586821...,{'melee/mario': {'melee/falco': [227.735426978...,"{'melee/mario': {'melee/falco': 3, 'melee/jigg...",{'melee/mario': {'melee/marth': <glicko2.glick...
2,4624,{'melee/jigglypuff': {'melee/pikachu': [2015-0...,{'melee/jigglypuff': {'melee/pikachu': [1747.3...,{'melee/jigglypuff': {'melee/pikachu': [253.40...,"{'melee/jigglypuff': {'melee/pikachu': 2}, 'me...",{'melee/zelda': {'melee/pikachu': <glicko2.gli...
3,4469,{'melee/falco': {'melee/marth': [2015-04-13 05...,{'melee/falco': {'melee/marth': [1747.31809422...,{'melee/falco': {'melee/marth': [253.404613850...,"{'melee/falco': {'melee/marth': 2, 'melee/pika...",{'melee/marth': {'melee/jigglypuff': <glicko2....
4,6189,{'melee/falco': {'melee/fox': [2015-04-13 05:0...,{'melee/falco': {'melee/fox': [1662.3108949761...,{'melee/falco': {'melee/fox': [290.31896467475...,"{'melee/falco': {'melee/fox': 2, 'melee/peach'...",{'melee/fox': {'melee/zelda': <glicko2.glicko2...


In [14]:
player_ratings_matchup_df.to_pickle('../player_ratings_matchup_data/player_ratings_matchup_tau_7_df.pkl')

In [15]:
loaded =  pd.read_pickle('../player_ratings_matchup_data/player_ratings_matchup_tau_7_df.pkl')
# loaded.head()

In [16]:
# im